The purpose of this notebook is to generate articles paragraph by paragraph using the trained models. The articles will be generated using the existing paragraphs from the articles themselves, continuously evaluating and adding paragraphs. The query that is passed to the model is a tokenized version of the of the article that has been built up to that point, starting with the first paragraph.

In [ ]:
! pip install trec-car-tools transformers
import torch
import sys
from torch.nn import functional, CosineEmbeddingLoss, BCEWithLogitsLoss, CosineSimilarity, BCELoss
from torch.utils.data.dataset import Dataset
from trec_car import read_data
from cbor import *
from transformers import BertModel, BertTokenizer, BertForNextSentencePrediction, Trainer, TrainingArguments
import random
# model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
file = 'fold-0-train.pages.cbor'

In [ ]:
pages = read_data.iter_pages(open(file, 'rb'))
page = next(pages)
my_model = BertModel.from_pretrained('drive/MyDrive/thesis_models/bert/checkpoint-5928')
bertmodel = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
# my model
corpus = page.get_text().split('\n')
cos = CosineSimilarity(dim=1, eps=1e-6)


output_file = 'out-batch-1.txt'
query = corpus.pop(0) # first para
out_text = query

pmodel_vals = []

# run each of the paragraphs through the model and save them
for p in corpus:
    pemb = tokenizer(p, return_tensors='pt', padding='max_length', max_length=128, truncation=True)['input_ids']
    pmodel = model(pemb)[0]
    pmodel_vals.append(pmodel)
    
# for each paragraph, compare it to the query and find the closest
while len(corpus) != 0:
  qemb = tokenizer(query, return_tensors='pt', padding='max_length', max_length=128, truncation=True)['input_ids']
  qout = model(qemb)[0]
  cos_vals = []
  
  for p in pmodel_vals:

    sim = cos(qout[0].type(torch.FloatTensor), p[0].type(torch.FloatTensor)) # similarity between the two output tensors
    cos_vals.append(sim.mean())

  next_para = corpus.pop(cos_vals.index(max(cos_vals)))
  pmodel_vals.pop(cos_vals.index(max(cos_vals)))
  query = query + next_para
  out_text = out_text + '\n' + next_para


with open(output_file, 'w') as f:
   print(out_text, file = f)


In [ ]:
# bert

corpus = page.get_text().split('\n')
cos = CosineSimilarity(dim=1, eps=1e-6)


bert_file = 'bert_out.txt'
query = corpus.pop(0) # first para
out_text = query

pmodel_vals = []

# run each of the paragraphs through the model and save them
for p in corpus:
    pemb = tokenizer(p, return_tensors='pt', padding='max_length', max_length=128, truncation=True)['input_ids']
    pmodel = model(pemb)[0]
    pmodel_vals.append(pmodel)
    
# for each paragraph, compare it to the query and find the closest
while len(corpus) != 0:
  qemb = tokenizer(query, return_tensors='pt', padding='max_length', max_length=128, truncation=True)['input_ids']
  qout = model(qemb)[0]
  cos_vals = []
  
  for p in pmodel_vals:

    sim = cos(qout[0].type(torch.FloatTensor), p[0].type(torch.FloatTensor)) # similarity between the two output tensors
    cos_vals.append(sim.mean())

  next_para = corpus.pop(cos_vals.index(max(cos_vals)))
  pmodel_vals.pop(cos_vals.index(max(cos_vals)))
  query = query + next_para
  out_text = out_text + '\n' + next_para

with open(bert_file, 'w') as f:
   print(out_text, file = f)

In [ ]:
#random baseline
corpus = page.get_text().split('\n')
cos = CosineSimilarity(dim=1, eps=1e-6)


random_file = 'random_out.txt'
query = corpus.pop(0) # first para

# for each paragraph, compare it to the query and find the closest
while len(corpus) != 0:
  next_para = corpus.pop(random.randrange(len(corpus)))
  query = query + '\n' + next_para

with open(random_file, 'w') as f:
   print(query, file = f)



tensor(-0.0009)